In [1]:
import pandas as pd

In [2]:
predictions = pd.read_csv(f"../data/predictions/predictions-{11}-{14}.csv")
features = pd.read_csv("../data/tsfeatures/tsfeatures.csv")

In [3]:
predictions.rename(columns={list(predictions)[0]:'date'}, inplace=True)
predictions['date'] = pd.to_datetime(predictions['date'], errors='coerce')

sales = pd.DataFrame(predictions.sales)

errors = predictions.drop(predictions.columns[[0, 1, 2]], axis=1)\
    .sub(sales.sales, axis=0)\
        .divide(sales.sales, axis=0) * 100

In [4]:
from helper import create_features

In [5]:
model_least_err = pd.DataFrame(errors.abs().idxmin(axis=1), columns=["model_w_least_error"])

errors_df = pd.concat([predictions[["store_item", "date"]], model_least_err], axis=1)

feats = features.drop(features.columns[[0, 3, 6, 7, 8]], axis=1)\
    .rename(columns={"unique_id":"store_item"})

errors_and_feats = pd.merge(errors_df, feats, right_on="store_item", left_on= "store_item")

error_time = pd.concat([errors_and_feats, create_features(predictions)], axis=1)\
    .dropna(axis = 1, how = 'all')\
    .dropna()
    
X, y = error_time.iloc[:, 3:], error_time.iloc[:, 2]

In [6]:
X.to_csv("../data/forecasts_for_ml/X.csv", index=False)
y.to_csv("../data/forecasts_for_ml/y.csv", index=False)